In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import subprocess
import shlex
import numpy as np
import pandas as pd

import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src.utils.file_handlers import group_dataset_files, read_dataset_files
from src.utils.operation_helpers import run_models, split_games
from src.utils.c_operation_helpers import  run_simulation
from src.utils.file_handlers import process_directory_real_data

### Example

Python

In [2]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

data, pi_values = read_dataset_files(grouped['00103'], data_dir, is_synthetic=False)

# train, test = train_test_split(data, train_size=.8)
train, test = split_games(data, .8)

df = run_models(train, test, pi_values)
print(df)


         model  log-likelihoods  leadership-log-likelihood
0        HO_BT        -2.744749                  -0.832101
1       HOL_BT        -2.742304                  -0.823259
2          BIN        -2.792092                  -0.843856
3         BINL        -2.789306                  -0.837697
4  Spring_Rank        -2.887182                  -0.849661
5    Page_Rank        -3.827274                  -1.095218
6   Point_Wise        -2.922654                  -0.844673


C 

In [3]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

edge_file = grouped['00103']['edges']
node_file = grouped['00103']['nodes']

edge_path = os.path.join(data_dir, edge_file)
node_path = os.path.join(data_dir, node_file)

print(edge_path)
print(node_path)

results = run_simulation(node_path, edge_path, is_synthetic=0)
print(results)

/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00103_edges.txt
/home/jackyeung99/senior_thesis/higher_order_ranking/datasets/Real_Data/00103_nodes.txt
  model  av_error   spearman     kendall     prior   HO_Like   HOL_Like  \
0    HO  0.359671  -0.140919  -0.0984426  -1.56191  -2.34644  -0.885774   
1   HOL  0.318249  0.0500658   0.0279165   -1.4955  -2.35228   -0.85317   
2   BIN  0.368074  -0.118072  -0.0755216  -1.59739  -2.37427  -0.882289   
3  BINL  0.332023   0.033475   0.0149868  -1.54735  -2.38077  -0.852221   

  iterations  
0          9  
1         10  
2         10  
3          9  


### Full Results

In [4]:
EXPERIMENT_PATH = os.path.join(repo_root, 'exp', 'ex04')
os.chdir(EXPERIMENT_PATH)

In [7]:
print(os.getcwd())

/home/jackyeung99/senior_thesis/higher_order_ranking/exp/ex04


In [8]:
process_directory_real_data(EXPERIMENT_PATH)

dataset-35_rep-16.csv
dataset-18_rep-35.csv
dataset-52_rep-34.csv
dataset-103_rep-49.csv
dataset-24_rep-42.csv
dataset-104_rep-12.csv
dataset-8_rep-9.csv
dataset-42_rep-7.csv
dataset-45_rep-4.csv
dataset-2_rep-7.csv
dataset-53_rep-38.csv
dataset-14_rep-40.csv
dataset-103_rep-36.csv
dataset-42_rep-15.csv
dataset-52_rep-39.csv
dataset-45_rep-23.csv
dataset-7_rep-36.csv
dataset-6_rep-10.csv
dataset-100_rep-37.csv
dataset-42_rep-45.csv
dataset-4_rep-42.csv
dataset-102_rep-31.csv
dataset-8_rep-33.csv
dataset-9_rep-33.csv
dataset-35_rep-29.csv
dataset-28_rep-30.csv
dataset-8_rep-29.csv
dataset-18_rep-10.csv
dataset-45_rep-16.csv
dataset-25_rep-35.csv
dataset-1_rep-36.csv
dataset-00014-epoch_9.csv


ValueError: too many values to unpack (expected 2)

In [7]:


def process_results():
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()
    leadership_log_likelihood_df = pd.read_csv(os.path.join(EXPERIMENT_PATH, 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()

    filtered_log_like = log_likelihood_df.drop(columns=['rep'])
    filtered_leader_like = leadership_log_likelihood_df.drop(columns=['rep'])
    
    log_like = subtract_columns(filtered_log_like, 3)
    leadership_log = subtract_columns(filtered_leader_like, 4)

    return log_like, leadership_log

def subtract_columns(df, compared_col):
    columns = df.columns.tolist()
    base_column = columns[compared_col] 
    print(base_column)
    for col in columns[1:]:
        
        df[col] = df[col] - df[base_column] 
    
    return df


def visualize_df(df, title):
    print(f"\n{title}\n")
    styled_df = df.style.set_table_styles(
    ).set_properties(**{
        'background-color': 'LightGray',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'left'
    })
    

    # print(df.to_latex(escape=True))
    display(styled_df)
   

In [ ]:


unique_datasets = pd.read_csv(os.path.join(os.getcwd(), 'results', 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['dataset_id'].isin(unique_datasets)].set_index('dataset_id')

domain_order = ["Election", "Sport", "Preferences", "Other"]
filtered_dataset_info['Domain'] = pd.Categorical(filtered_dataset_info['Domain'], categories=domain_order, ordered=True)

filtered_dataset_info = filtered_dataset_info.sort_values(by=['Domain', 'K1', 'K2', 'Name'])
visualize_df(filtered_dataset_info, 'Datasets Tested')

EmptyDataError: No columns to parse from file

In [9]:
log_like, leadership_log = process_results(os.path.join(os.getcwd(), 'results'))

def rename_df(df):
    merged_df = df.merge(dataset_info, left_on='dataset', right_on='dataset_id').drop(columns=['dataset_id','dataset'])
    # merged_df = merged_df.rename(columns={'Name': 'dataset_name'}).set_index('dataset_name')

    domain_order = ["Election", "Sport", "Preferences", "Other"]
    merged_df['Domain'] = pd.Categorical(merged_df['Domain'], categories=domain_order, ordered=True)
    merged_df = merged_df.sort_values(by=['Domain', 'K1', 'K2', 'Name'])
    merged_df = merged_df[['Domain', 'Name', 'BT','BT_leadership', 'HO_BT', 'HOL_BT', 'Spring_Rank', 'Spring_Rank_Leadership', 'Page_Rank','Page_Rank_Leadership', 'Point_Wise']]

    return merged_df

FileNotFoundError: [Errno 2] No such file or directory: '/home/jackyeung99/senior_thesis/higher_order_ranking/exp/ex03/results/log_likelihood_summary.csv'

In [ ]:

log_like = rename_df(log_like)
visualize_df(log_like, 'log likelihood')

In [ ]:
leadership_log = rename_df(leadership_log)
visualize_df(leadership_log, 'Leadership log likelihood')